In [1]:
from agenticcore.chatbot.services import ChatBot
bot = ChatBot()
print(bot.reply("Testing from notebook"))


{'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5}


# 2) Config: choose backend URL and provider (HF/Azure/etc.)

In [2]:
import os

# Point to your FastAPI server (change if needed)
import os

# Default backend URL (can be overridden later via the widget)
BACKEND_URL = os.environ.get("BACKEND_URL", "http://127.0.0.1:8000")

# Provider hint (optional; providers_unified auto-detects if keys exist)
# Examples:
# os.environ["AI_PROVIDER"] = "hf"
# os.environ["HF_API_KEY"] = "hf_XXXXXXXX..."   # if using Hugging Face
# os.environ["MICROSOFT_AI_SERVICE_ENDPOINT"] = "https://<name>.cognitiveservices.azure.com/"
# os.environ["MICROSOFT_AI_API_KEY"] = "<your-azure-key>"

BACKEND_URL


'http://127.0.0.1:8000'

# 3) Helper functions (API + Library paths)

In [3]:
import os
import json
import requests
from typing import Dict, Any

# Default backend URL
BACKEND_URL = os.environ.get("BACKEND_URL", "http://127.0.0.1:8000")

def send_via_api(message: str, url: str = BACKEND_URL) -> Dict[str, Any]:
    """POST to FastAPI /chatbot/message. Returns dict with reply/sentiment/confidence."""
    u = url.rstrip("/") + "/chatbot/message"
    r = requests.post(u, json={"message": message}, timeout=20)
    r.raise_for_status()
    return r.json()

def send_via_library(message: str) -> Dict[str, Any]:
    """Call ChatBot() directly inside this kernel."""
    from agenticcore.chatbot.services import ChatBot
    return ChatBot().reply(message)

def health(url: str = BACKEND_URL) -> Dict[str, Any]:
    r = requests.get(url.rstrip("/") + "/health", timeout=10)
    r.raise_for_status()
    return r.json()


# 4) Minimal UI (ipywidgets)

In [4]:
import ipywidgets as W
from IPython.display import display, HTML, clear_output

mode = W.ToggleButtons(
    options=[("API", "api"), ("Library", "lib")],
    value="api",
    description="Route:",
)
backend = W.Text(value=BACKEND_URL, placeholder="http://127.0.0.1:8000", description="Backend:", layout=W.Layout(width="60%"))
save_btn = W.Button(description="Save", button_style="info")
msg = W.Text(placeholder="Type a message…", description="You:", layout=W.Layout(width="60%"))
send_btn = W.Button(description="Send", button_style="primary")
cap_btn = W.Button(description="Capabilities", tooltip="Show ChatBot capabilities")
out = W.Output()

def on_save(_):
    os.environ["BACKEND_URL"] = backend.value.strip()
    with out:
        print(f"[config] BACKEND_URL = {os.environ['BACKEND_URL']}")

def on_send(_):
    text = msg.value.strip()
    if not text:
        with out:
            print("[warn] Please enter some text.")
        return
    try:
        if mode.value == "api":
            data = send_via_api(text, backend.value.strip())
        else:
            data = send_via_library(text)
        with out:
            print(json.dumps(data, indent=2, ensure_ascii=False))
    except Exception as e:
        with out:
            print(f"[error] {e}")

def on_caps(_):
    try:
        # Prefer library capabilities; keeps working even if API is down
        from agenticcore.chatbot.services import ChatBot
        data = ChatBot().capabilities()
        with out:
            print(json.dumps({"capabilities": data}, indent=2))
    except Exception as e:
        with out:
            print(f"[error capabilities] {e}")

save_btn.on_click(on_save)
send_btn.on_click(on_send)
cap_btn.on_click(on_caps)

display(W.HBox([mode, backend, save_btn]))
display(W.HBox([msg, send_btn, cap_btn]))
display(out)

# Optional visual hint
display(HTML("""
<div style="margin-top:8px;opacity:.8">
  Tip: API path requires your FastAPI server running at /chatbot/message.
  Switch to <b>Library</b> mode for offline tests.
</div>
"""))


Output()

# 5) Batch test cell (multi-prompt, tabular)

In [5]:
import pandas as pd

tests = [
    "I absolutely love this project!",
    "This is awful and broken.",
    "Can you list your capabilities?",
    "",  # malformed/empty
]

rows = []
for t in tests:
    try:
        data = send_via_api(t, backend.value.strip()) if mode.value == "api" else send_via_library(t)
        rows.append({"message": t, **data})
    except Exception as e:
        rows.append({"message": t, "reply": f"(error) {e}", "sentiment": None, "confidence": None})

df = pd.DataFrame(rows)
df


,message,reply,sentiment,confidence
0,I absolutely love this project!,(error) 404 Client Error: Not Found for url: h...,None,None
1,This is awful and broken.,(error) 404 Client Error: Not Found for url: h...,None,None
2,Can you list your capabilities?,(error) 404 Client Error: Not Found for url: h...,None,None
3,,(error) 404 Client Error: Not Found for url: h...,None,None


# 6) Health check + quick assertions

In [11]:
try:
    print("Health:", health(backend.value.strip()))
except Exception as e:
    print("Health check failed:", e)

# Simple acceptance checks
sample = send_via_library("hello")
assert all(k in sample for k in ("reply", "sentiment", "confidence"))
print("Library OK:", sample)

sample_api = send_via_api("hello from api", backend.value.strip())
assert all(k in sample_api for k in ("reply", "sentiment", "confidence"))
print("API OK:", sample_api)


Health: {'ok': True, 'version': '0.3.0', 'time': 1757798428}
Library OK: {'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5}
API OK: {'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5, 'thread': None}


In [12]:
import requests, os, json
BACKEND_URL = os.environ.get("BACKEND_URL", "http://127.0.0.1:8000")
routes = requests.get(BACKEND_URL.rstrip("/") + "/openapi.json", timeout=10).json()["paths"]
print(json.dumps(list(routes.keys())[:20], indent=2))


[
  "/health",
  "/status",
  "/chatbot/message",
  "/ui"
]


In [13]:
send_via_api("hello from api", BACKEND_URL.strip())


{'reply': 'Noted. The sentiment appears neutral.',
 'sentiment': 'neutral',
 'confidence': 0.5,
 'thread': None}

In [14]:
print("Health:", health(BACKEND_URL))
sample = send_via_library("hello")
print("Library OK:", sample)

sample_api = send_via_api("hello from api", BACKEND_URL)
print("API OK:", sample_api)


Health: {'ok': True, 'version': '0.3.0', 'time': 1757798440}
Library OK: {'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5}
API OK: {'reply': 'Noted. The sentiment appears neutral.', 'sentiment': 'neutral', 'confidence': 0.5, 'thread': None}


In [16]:
# Pick a clean port to avoid collisions (e.g., 8077)
uvicorn backend.app.main:app --reload --port 8077 --app-dir .


SyntaxError: invalid syntax (3247471142.py, line 2)